In [ ]:
import numpy as np
from layers.layerBase import LayerBase as IDMapping
from layers.affine import Affine
from layers.batchNormalization import BatchNormalization
from layers.relu import Relu
from layers.dropout import Dropout
from functions.UtilityFunctions import scale
from models.model import Model
from models.trainer import Trainer
from models.optimizer import SGD, AdaGrad, Momentum, RMSprop, Adam
from functions.EvaluationFunctions import Metrics, TYPE
from functions.LossFunctions import linear

In [ ]:
import pandas as pd
columns = {'CRIM':float, 'ZN':float, 'INDUS':float, 'CHAS':int, 'NOX':float, 'RM':float, 'AGE':float, 'DIS':float, 'RAD':object, 'TAX':int, 'PTRATIO':float, 'B':float, 'LSTAT':float, 'TARGET':float}
df = pd.read_csv('./housing.data', header=None, sep='\s+', na_values='na', names=columns.keys(), dtype=columns)
y = df['TARGET']
tp = df[['CHAS','RAD']]
df = df.drop(['TARGET','CHAS','RAD'], axis=1)
df = (df - df.mean()) / df.std(ddof=0)
tp = pd.get_dummies(tp, dummy_na=False, columns=['RAD'])
df = pd.concat([df,tp], axis=1, sort=False)
df.head()

In [ ]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(n_splits=1, train_size=0.7, random_state=0)
train_idx, test_idx = next(ss.split(df, y))
df = df.to_numpy()
y = y.to_numpy()
x_train, x_test, t_train, t_test = df[train_idx], df[test_idx], y[train_idx], y[test_idx]

In [ ]:
#モデル生成
m = Model()
m.append(Affine(scale(21) * np.random.randn(21, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 1), np.zeros(1)))
m.append(IDMapping())
m.summary(np.zeros_like(x_train[[0]]))

In [ ]:
#学習
t = Trainer(
    model=m,
    optimizer=AdaGrad(lr=0.1),
    loss=linear,
    metrics=Metrics(TYPE.R2)
)
t.fit(x_train, t_train, 20, 50)

In [ ]:
#検証
t.predict(x_test, t_test)